### 🚀 Noticias Universidad Panamericana

Este ejemplo busca obtener todas las noticias de la Universidad Panamericana. Paso por paso, se creará un spider para obtener las noticias y seguirá los siguientes pasos:

1. Crear un spider
2. Obtener todas las noticias
3. Obtener el enlace de cada noticia
4. Obtener el contenido de cada noticia
5. Guardar el contenido de cada noticia en en memoria
6. Usar pandas para guardar el contenido de cada noticia en un archivo csv.

In [31]:
%pip install pandas requests beautifulsoup4 pydantic openpyxl

  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ------ -------------------------------- 41.0/250.9 kB 326.8 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 326.8 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/250.9 kB 607.9 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/250.9 kB 607.9 kB/s eta 0:00:01
   --------------------- ---------------- 143.4/250.9 k


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pydantic import BaseModel
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Optional

In [ ]:
class NewsIndex(BaseModel):
    title: str
    url: str
    category: str
    date: str


class NewsPost(BaseModel):
    title: str
    url: Optional[str]
    category: str
    date: str
    content: str
    image: str

In [ ]:
def get_news_page(page=1) -> str:
    url = f"https://www.up.edu.mx/en/tema/noticias/page/{page}"
    response = requests.get(url)

    response.raise_for_status()
    return response.text

def get_total_pages():
    html = get_news_page(page=1)
    soup = BeautifulSoup(html, "html.parser")

    pagination = soup.select_one("nav.wp-block-query-pagination")

    if not pagination:
        return 1

    pages = []
    for a in pagination.select("a.page-numbers"):
        text = a.get_text(strip=True)
        if text.isdigit():
            pages.append(int(text))

    return max(pages) if pages else 1


def parse_news_page(html) -> list[NewsIndex]:
    soup = BeautifulSoup(html, "html.parser")

    blocks = soup.select(".wp-block-group.wp-container-content-9e2f13cb")

    results = []

    for block in blocks:
        category = block.select_one(".taxonomy-category a")
        category = category.get_text(strip=True) if category else None

        date = block.select_one(".wp-block-post-date time")
        date = date.get_text(strip=True) if date else None

        title_link = block.select_one(".wp-block-post-title a")
        if title_link:
            title = title_link.get_text(strip=True)
            url = title_link["href"]

        if not title or not url:
            continue

        results.append(NewsIndex(
            title=title,
            url=url,
            category=category,
            date=date
        ))

    return results


def spider_all_news_index():
    print("✨ Scraping all news index")
    total_pages = get_total_pages()
    print("✨ Total pages:", total_pages)

    all_news = []

    # TODO: Remove this. Only for testing
    total_pages = 1

    for page in range(1, total_pages + 1):
        print(f"⬇️ Getting page #{page}")
        html = get_news_page(page)
        print(f"📄 Parsing page #{page}")
        news = parse_news_page(html)
        print(f"📄 Saving {len(news)} news")
        all_news.extend(news)

    return all_news

In [7]:
news_index = spider_all_news_index()
print("🚀 news_index total:", len(news_index))
print("🚀 news_index:", news_index)


✨ Scraping all news index
✨ Total pages: 89
⬇️ Getting page #1
📄 Parsing page #1
📄 Saving 11 news
🚀 news_index total: 11
🚀 news_index: [NewsIndex(title='UP fortalece su oferta internacional con Doble Titulación en alianza con CityU', url='https://www.up.edu.mx/noticias/internacionalizacion/up-fortalece-su-oferta-internacional-con-doble-titulacion-en-alianza-con-cityu/', category='Internacionalización', date='Dic 8, 2025'), NewsIndex(title='UP Aguascalientes lanza Ingeniería en Energía y Sistemas Inteligentes: una apuesta estratégica hacia el futuro energético', url='https://www.up.edu.mx/noticias/excelencia-academica/up-aguascalientes-lanza-ingenieria-en-energia-y-sistemas-inteligentes-una-apuesta-estrategica-hacia-el-futuro-energetico/', category='Excelencia Académica', date='Dic 8, 2025'), NewsIndex(title='IA y Educación Humanista: la visión del Sistema UP-IPADE en la FIL Guadalajara', url='https://www.up.edu.mx/noticias/arte-cultura-y-difusion/ia-y-educacion-humanista-la-vision-del-

In [9]:
print("🚀 News index total:", len(news_index))
for news in news_index:
    print("⬇️ Getting news", {
        "title": news.title,
        "category": news.category,
        "date": news.date,
        "url": news.url
    })

🚀 News index total: 11
⬇️ Getting news {'title': 'UP fortalece su oferta internacional con Doble Titulación en alianza con CityU', 'category': 'Internacionalización', 'date': 'Dic 8, 2025', 'url': 'https://www.up.edu.mx/noticias/internacionalizacion/up-fortalece-su-oferta-internacional-con-doble-titulacion-en-alianza-con-cityu/'}
⬇️ Getting news {'title': 'UP Aguascalientes lanza Ingeniería en Energía y Sistemas Inteligentes: una apuesta estratégica hacia el futuro energético', 'category': 'Excelencia Académica', 'date': 'Dic 8, 2025', 'url': 'https://www.up.edu.mx/noticias/excelencia-academica/up-aguascalientes-lanza-ingenieria-en-energia-y-sistemas-inteligentes-una-apuesta-estrategica-hacia-el-futuro-energetico/'}
⬇️ Getting news {'title': 'IA y Educación Humanista: la visión del Sistema UP-IPADE en la FIL Guadalajara', 'category': 'Arte cultura y difusión', 'date': 'Dic 5, 2025', 'url': 'https://www.up.edu.mx/noticias/arte-cultura-y-difusion/ia-y-educacion-humanista-la-vision-del-si

In [10]:
def spider_all_news_index_parallel() -> List[NewsIndex]:
    print("✨ Scraping all news index")
    total_pages = get_total_pages()
    print("✨ Total pages:", total_pages)

    all_news = []

    # TODO: Remove this. Only for testing
    total_pages = 10

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for page in range(1, total_pages + 1):
            print(f"⬇️ Getting page #{page}")
            futures.append(executor.submit(get_news_page, page))

        for future in as_completed(futures):
            html = future.result()
            print(f"📄 Parsing page #{page}")
            news = parse_news_page(html)
            print(f"📄 Saving {len(news)} news")
            all_news.extend(news)

    return all_news


news_index_parallel = spider_all_news_index_parallel()
print("🚀 news_index total:", len(news_index_parallel))
print("🚀 news_index:", news_index_parallel)

✨ Scraping all news index
✨ Total pages: 89
⬇️ Getting page #1
⬇️ Getting page #2
⬇️ Getting page #3
⬇️ Getting page #4
⬇️ Getting page #5
⬇️ Getting page #6
⬇️ Getting page #7
⬇️ Getting page #8
⬇️ Getting page #9
⬇️ Getting page #10
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
📄 Parsing page #10
📄 Saving 11 news
🚀 news_index total: 110
🚀 news_index: [NewsIndex(title='Halberd Studios: emprendimiento UP que conquista la industria global de videojuegos', url='https://www.up.edu.mx/noticias/innovacion-y-emprendimiento/halberd-studios-emprendimiento-up-que-conquista-la-industria-global-de-videojuegos/', category='Innovación y emprendimiento', date='Sep 24, 2025'), NewsIndex(title='UP refuerza su compromiso ambienta

In [ ]:
def get_news_post_page(url) -> str:
    response = requests.get(url)

    response.raise_for_status()
    return response.text


def parse_news_post_page(html, news_index: NewsIndex) -> NewsPost:
    soup = BeautifulSoup(html, "html")

    title_el = soup.select_one(".wp-block-post-title")
    title = title_el.get_text(strip=True) if title_el else None

    cat_el = soup.select_one(".taxonomy-category a")
    category = cat_el.get_text(strip=True) if cat_el else None

    date_el = soup.select_one(".wp-block-post-date time")
    date = date_el.get_text(strip=True) if date_el else None

    img_el = soup.select_one(".wp-block-post-featured-image img")
    image = img_el["src"] if img_el and img_el.has_attr("src") else None

    block = soup.select_one(".wp-block-post-content, .entry-content")
    content = block.get_text(strip=True, separator=" ") if block else None

    return NewsPost(
        title=title,
        url=news_index.url,
        content=content,
        image=image,
        category=category,
        date=date
    )


def scrape_news_post(news_index: List[NewsIndex]) -> list[NewsPost]:
    all_news = []

    print("✨ Scraping news posts...")

    for index in news_index:
        print(f"⬇️ Getting page for {index.title}")
        html = get_news_post_page(index.url)

        print(f"📄 Parsing page for {index.title}")
        post = parse_news_post_page(html, index)

        print(f"📄 Saved {index.title}")
        all_news.append(post)

    return all_news

In [13]:
news_posts = scrape_news_post(
    news_index_parallel[0:2]
)
print("🚀 News posts total: ", len(news_posts))


🚀 News posts total:  2
🚀 News posts example:  title='Halberd Studios: emprendimiento UP que conquista la industria global de videojuegos' url='https://www.up.edu.mx/noticias/innovacion-y-emprendimiento/halberd-studios-emprendimiento-up-que-conquista-la-industria-global-de-videojuegos/' category='Innovación y emprendimiento' date='septiembre 24, 2025' content='19 de septiembre de 2025.-En el más reciente episodio dePanamericana Emprende,podcast de laUniversidad Panamericanadedicado a las historias de emprendimiento e innovación de negocios, se exploró el camino deHalberd Studios, un estudio de desarrollo de videojuegos nacido en las aulas de la Panamericana y que hoy es unreferente de talento mexicano a nivel internacional.Para hablar de éste, se contó con elDr. Jesús Gómez Romero Bórquez, socio del estudio y director de la Carrera de Ingeniería en Animación Digital en campus Guadalajara, y elLic. Raúl Bonillas, director operativo de Halberd Studios, como invitados. Ambos compartieron c

In [22]:
e_post = news_posts[0]

print(
    "🚀 News posts example: ",
    json.dumps({
        "title": e_post.title,
        "image": e_post.image,
        "url": e_post.url,
        "category": e_post.category,
        "date": e_post.date,
        "content": e_post.content,
    }, indent=2, ensure_ascii=False)
)

🚀 News posts example:  {
  "title": "Halberd Studios: emprendimiento UP que conquista la industria global de videojuegos",
  "image": "https://www.up.edu.mx/wp-content/uploads/2025/09/IMG_0147-scaled.jpg",
  "url": "https://www.up.edu.mx/noticias/innovacion-y-emprendimiento/halberd-studios-emprendimiento-up-que-conquista-la-industria-global-de-videojuegos/",
  "category": "Innovación y emprendimiento",
  "date": "septiembre 24, 2025",
  "content": "19 de septiembre de 2025.-En el más reciente episodio dePanamericana Emprende,podcast de laUniversidad Panamericanadedicado a las historias de emprendimiento e innovación de negocios, se exploró el camino deHalberd Studios, un estudio de desarrollo de videojuegos nacido en las aulas de la Panamericana y que hoy es unreferente de talento mexicano a nivel internacional.Para hablar de éste, se contó con elDr. Jesús Gómez Romero Bórquez, socio del estudio y director de la Carrera de Ingeniería en Animación Digital en campus Guadalajara, y elLic.

In [26]:
def scrape_news_post_parallel(news_index: List[NewsIndex]) -> list[NewsPost]:
    all_news = []

    print("✨ Scraping news posts...")

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        futures_map = {}
        for index in news_index:
            print(f"⬇️ Getting page {index.title}")
            future = executor.submit(get_news_post_page, index.url)
            futures.append(future)
            futures_map[future] = index

        for future in as_completed(futures):
            html = future.result()
            print(f"📄 Parsing page for {index.title}")
            index = futures_map[future]
            post = parse_news_post_page(html, index)
            print(f"📄 Saved {index.title}")
            all_news.append(post)

    return all_news


news_posts_parallel = scrape_news_post_parallel(news_index_parallel[0:50])
print("🚀 News posts total: ", len(news_posts_parallel))

✨ Scraping news posts...
⬇️ Getting page Halberd Studios: emprendimiento UP que conquista la industria global de videojuegos
⬇️ Getting page UP refuerza su compromiso ambiental con séptima reforestación
⬇️ Getting page UP Aguascalientes inaugura el Centro Institucional de Innovación Educativa
⬇️ Getting page Universidad Panamericana continúa siendo hogar de sus colaboradores jubilados
⬇️ Getting page UP impulsa la justicia climática con proyecto trinacional sobre sistemas alimentarios indígenas
⬇️ Getting page Universidad Panamericana presente en el 8vo Encuentro con Premios Nobel de Economía
⬇️ Getting page UP presenta Ejecutar ejecutando. El arte de hacer que las cosas sucedan
⬇️ Getting page La Universidad Panamericana inaugura la era EVOLVE: revoluciona su ecosistema digital para una educación del futuro
⬇️ Getting page Reconocen talento universitario con Beca Mtro. Francisco Benítez
⬇️ Getting page UP genera conciencia sobre la prevención del suicidio con conferencia especial
⬇️ G

In [32]:
df = pd.DataFrame([
    {
        "title": post.title,
        "url": post.url,
        "date": post.date,
        "content": post.content,
        "image": post.image,
        "category": post.category,
    }
    for post in news_posts_parallel
])
print(df.head())

df.to_excel("news_posts.xlsx", index=False)

                                               title  \
0  UP Aguascalientes inaugura el Centro Instituci...   
1  La Universidad Panamericana inaugura la era EV...   
2  UP impulsa la justicia climática con proyecto ...   
3  UP genera conciencia sobre la prevención del s...   
4  Halberd Studios: emprendimiento UP que conquis...   

                                                 url                 date  \
0  https://www.up.edu.mx/noticias/innovacion-y-em...  septiembre 22, 2025   
1  https://www.up.edu.mx/noticias/innovacion-y-em...  septiembre 17, 2025   
2  https://www.up.edu.mx/noticias/investigacion/u...  septiembre 19, 2025   
3  https://www.up.edu.mx/noticias/enfoque-humano/...  septiembre 10, 2025   
4  https://www.up.edu.mx/noticias/innovacion-y-em...  septiembre 24, 2025   

                                             content  \
0  Aguascalientes., Aguascalientes, 22 de septiem...   
1  Ciudad de México, 10 de septiembre de 2025.- C...   
2  Ciudad de México, 3 de septie